# Chapter 4 - Exercise 7: Bioink Rheology and Printability

## Learning Objectives
- Understand rheological properties critical for bioprinting
- Characterize shear-thinning and yield stress behavior
- Apply power-law and Herschel-Bulkley models
- Analyze thixotropy and time-dependent recovery
- Predict shape fidelity and printability windows
- Optimize bioink formulations for extrusion and inkjet

## Background

**Bioink rheology** determines printability and shape fidelity. The ideal bioink must:
- **Flow easily during printing** (low viscosity under shear)
- **Hold shape after deposition** (high viscosity at rest)
- **Recover quickly** (rapid gelation/crosslinking)
- **Protect cells** (shear-thinning behavior)

### Key Rheological Models:

**Newtonian Fluids** (constant viscosity):
$$\tau = \eta \dot{\gamma}$$

**Power-Law Model** (shear-thinning):
$$\tau = K \dot{\gamma}^n$$

where:
- n < 1: shear-thinning (pseudoplastic)
- n = 1: Newtonian
- n > 1: shear-thickening (rare)

**Herschel-Bulkley Model** (yield stress + shear-thinning):
$$\tau = \tau_y + K \dot{\gamma}^n$$

**Apparent Viscosity**:
$$\eta_a = \frac{\tau}{\dot{\gamma}} = K \dot{\gamma}^{n-1}$$

### Critical Parameters:

**For Extrusion**:
- Viscosity at low shear (storage): 10³-10⁶ mPa·s
- Viscosity at high shear (printing): 10²-10³ mPa·s
- Yield stress: 10-500 Pa
- Shear-thinning index: n = 0.3-0.7
- Recovery time: <60 seconds

**For Inkjet**:
- Viscosity: 1-10 mPa·s (nearly Newtonian)
- Z-number: 1 < Z < 10
- No yield stress

### Printability Metrics:

**Shape Fidelity (Pr)** - Printability number:
$$Pr = \frac{\tau_y}{\rho g h}$$

- Pr >> 1: excellent shape retention
- Pr ≈ 1: marginal stability
- Pr << 1: collapse/spreading

## Setup: Install and Import Libraries

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install numpy matplotlib pandas seaborn scipy plotly -q

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import curve_fit
from scipy.integrate import odeint
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ All libraries loaded successfully!")
print("Ready to explore bioink rheology!")

## Part 1: Rheological Model Functions

In [ ]:
def newtonian_model(shear_rate, viscosity):
    """
    Newtonian fluid model: τ = η·γ̇
    
    Parameters:
    -----------
    shear_rate : float or array
        Shear rate (1/s)
    viscosity : float
        Dynamic viscosity (Pa·s)
    
    Returns:
    --------
    shear_stress : float or array
        Shear stress (Pa)
    """
    return viscosity * shear_rate

def power_law_model(shear_rate, K, n):
    """
    Power-law model: τ = K·γ̇ⁿ
    
    Parameters:
    -----------
    shear_rate : float or array
        Shear rate (1/s)
    K : float
        Consistency index (Pa·sⁿ)
    n : float
        Flow behavior index (dimensionless)
    
    Returns:
    --------
    shear_stress : float or array
        Shear stress (Pa)
    """
    return K * (shear_rate ** n)

def herschel_bulkley_model(shear_rate, tau_y, K, n):
    """
    Herschel-Bulkley model: τ = τ_y + K·γ̇ⁿ
    
    Parameters:
    -----------
    shear_rate : float or array
        Shear rate (1/s)
    tau_y : float
        Yield stress (Pa)
    K : float
        Consistency index (Pa·sⁿ)
    n : float
        Flow behavior index (dimensionless)
    
    Returns:
    --------
    shear_stress : float or array
        Shear stress (Pa)
    """
    return tau_y + K * (shear_rate ** n)

def apparent_viscosity(shear_rate, K, n):
    """
    Calculate apparent viscosity for power-law fluid.
    
    Parameters:
    -----------
    shear_rate : float or array
        Shear rate (1/s)
    K : float
        Consistency index (Pa·sⁿ)
    n : float
        Flow behavior index (dimensionless)
    
    Returns:
    --------
    eta_apparent : float or array
        Apparent viscosity (Pa·s)
    """
    return K * (shear_rate ** (n - 1))

def calculate_printability_number(yield_stress, density, height):
    """
    Calculate printability number (shape fidelity metric).
    
    Parameters:
    -----------
    yield_stress : float
        Yield stress τ_y (Pa)
    density : float
        Bioink density ρ (kg/m³)
    height : float
        Structure height h (m)
    
    Returns:
    --------
    Pr : float
        Printability number (dimensionless)
    """
    g = 9.81  # m/s²
    Pr = yield_stress / (density * g * height)
    return Pr

def thixotropic_recovery(time, tau_initial, tau_final, recovery_time_constant):
    """
    Model thixotropic recovery (exponential).
    
    Parameters:
    -----------
    time : float or array
        Time after shear cessation (s)
    tau_initial : float
        Yield stress immediately after shear (Pa)
    tau_final : float
        Equilibrium yield stress (Pa)
    recovery_time_constant : float
        Time constant for recovery (s)
    
    Returns:
    --------
    tau : float or array
        Yield stress at time t (Pa)
    """
    return tau_final - (tau_final - tau_initial) * np.exp(-time / recovery_time_constant)

def fit_power_law(shear_rates, shear_stresses):
    """
    Fit power-law model to experimental data.
    
    Parameters:
    -----------
    shear_rates : array
        Measured shear rates (1/s)
    shear_stresses : array
        Measured shear stresses (Pa)
    
    Returns:
    --------
    K, n : float
        Fitted parameters
    """
    # Log-log linear fit: log(τ) = log(K) + n·log(γ̇)
    log_gamma = np.log10(shear_rates)
    log_tau = np.log10(shear_stresses)
    
    # Linear regression
    coeffs = np.polyfit(log_gamma, log_tau, 1)
    n = coeffs[0]
    log_K = coeffs[1]
    K = 10 ** log_K
    
    return K, n

def assess_printability_extrusion(viscosity_low_shear, viscosity_high_shear, 
                                  yield_stress, recovery_time):
    """
    Assess bioink printability for extrusion.
    
    Parameters:
    -----------
    viscosity_low_shear : float
        Viscosity at 0.1 1/s (Pa·s)
    viscosity_high_shear : float
        Viscosity at 100 1/s (Pa·s)
    yield_stress : float
        Yield stress (Pa)
    recovery_time : float
        Time to 90% recovery (s)
    
    Returns:
    --------
    dict : Assessment with score and recommendations
    """
    assessment = {
        'score': 0,
        'rating': '',
        'issues': [],
        'recommendations': []
    }
    
    score = 0
    
    # Check viscosity range (want high at rest, low during printing)
    viscosity_ratio = viscosity_low_shear / viscosity_high_shear
    
    if 10 < viscosity_ratio < 1000:
        score += 30
    elif viscosity_ratio > 5:
        score += 15
        assessment['issues'].append('Moderate shear-thinning')
    else:
        assessment['issues'].append('Insufficient shear-thinning')
        assessment['recommendations'].append('Increase polymer concentration or add thickener')
    
    # Check absolute viscosities
    if 100 < viscosity_low_shear < 10000:
        score += 25
    else:
        if viscosity_low_shear < 100:
            assessment['issues'].append('Too fluid at rest - poor shape fidelity')
        else:
            assessment['issues'].append('Too viscous - may clog nozzle')
    
    if 0.1 < viscosity_high_shear < 10:
        score += 20
    else:
        if viscosity_high_shear < 0.1:
            assessment['issues'].append('Too low viscosity during printing')
        else:
            assessment['issues'].append('Too high viscosity - high pressure needed')
    
    # Check yield stress
    if 50 < yield_stress < 500:
        score += 15
    elif 10 < yield_stress < 50:
        score += 10
        assessment['issues'].append('Low yield stress - shape may collapse')
    else:
        if yield_stress < 10:
            assessment['issues'].append('Insufficient yield stress for self-support')
            assessment['recommendations'].append('Increase crosslinking or add nano-particles')
        else:
            assessment['issues'].append('Very high yield stress - may resist flow')
    
    # Check recovery time
    if recovery_time < 60:
        score += 10
    elif recovery_time < 120:
        score += 5
    else:
        assessment['issues'].append('Slow recovery - structure may deform')
        assessment['recommendations'].append('Add rapid crosslinking mechanism')
    
    # Rating
    if score >= 80:
        assessment['rating'] = 'Excellent'
    elif score >= 60:
        assessment['rating'] = 'Good'
    elif score >= 40:
        assessment['rating'] = 'Marginal'
    else:
        assessment['rating'] = 'Poor'
    
    assessment['score'] = score
    
    return assessment

print("✓ Rheological model functions defined!")
print("\nKey functions:")
print("  • newtonian_model()")
print("  • power_law_model()")
print("  • herschel_bulkley_model()")
print("  • apparent_viscosity()")
print("  • calculate_printability_number()")
print("  • thixotropic_recovery()")
print("  • assess_printability_extrusion()")

## Part 2: Visualize Rheological Models

In [ ]:
# Create shear rate range
shear_rates = np.logspace(-1, 3, 200)  # 0.1 to 1000 1/s

# Define example bioinks
bioinks = {
    'Newtonian (water)': {
        'model': 'newtonian',
        'params': {'viscosity': 0.001},  # Pa·s
        'color': '#3498db'
    },
    'Alginate (weak)': {
        'model': 'power_law',
        'params': {'K': 0.5, 'n': 0.6},
        'color': '#2ecc71'
    },
    'GelMA': {
        'model': 'power_law',
        'params': {'K': 2, 'n': 0.4},
        'color': '#f39c12'
    },
    'Alginate-Gelatin': {
        'model': 'herschel_bulkley',
        'params': {'tau_y': 50, 'K': 5, 'n': 0.5},
        'color': '#e74c3c'
    }
}

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Shear stress vs shear rate
for bioink_name, props in bioinks.items():
    if props['model'] == 'newtonian':
        tau = newtonian_model(shear_rates, props['params']['viscosity'])
    elif props['model'] == 'power_law':
        tau = power_law_model(shear_rates, props['params']['K'], props['params']['n'])
    else:  # herschel_bulkley
        tau = herschel_bulkley_model(shear_rates, props['params']['tau_y'],
                                     props['params']['K'], props['params']['n'])
    
    ax1.loglog(shear_rates, tau, linewidth=3, color=props['color'],
              label=bioink_name)

# Mark typical printing shear rates
ax1.axvspan(10, 1000, alpha=0.2, color='green', label='Extrusion range')
ax1.axvline(0.1, color='red', linestyle='--', linewidth=2, alpha=0.7,
           label='At rest (storage)')

ax1.set_xlabel('Shear Rate, γ̇ (1/s)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Shear Stress, τ (Pa)', fontsize=12, fontweight='bold')
ax1.set_title('Flow Curves: τ vs γ̇', fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax1.grid(True, alpha=0.3, which='both')

# Plot 2: Apparent viscosity vs shear rate
for bioink_name, props in bioinks.items():
    if props['model'] == 'newtonian':
        eta = np.ones_like(shear_rates) * props['params']['viscosity']
    elif props['model'] == 'power_law':
        eta = apparent_viscosity(shear_rates, props['params']['K'], props['params']['n'])
    else:  # herschel_bulkley
        tau = herschel_bulkley_model(shear_rates, props['params']['tau_y'],
                                     props['params']['K'], props['params']['n'])
        eta = tau / shear_rates
    
    ax2.loglog(shear_rates, eta * 1000, linewidth=3, color=props['color'],
              label=bioink_name)  # Convert to mPa·s

ax2.axvspan(10, 1000, alpha=0.2, color='green')
ax2.set_xlabel('Shear Rate, γ̇ (1/s)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Apparent Viscosity, η (mPa·s)', fontsize=12, fontweight='bold')
ax2.set_title('Viscosity Curves\nShear-Thinning Behavior', fontsize=13, fontweight='bold')
ax2.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax2.grid(True, alpha=0.3, which='both')

# Plot 3: Shear-thinning index comparison
n_values = [0.3, 0.5, 0.7, 0.9, 1.0]
K_fixed = 1.0
colors_n = ['#8e44ad', '#3498db', '#2ecc71', '#f39c12', '#e74c3c']

for n, color in zip(n_values, colors_n):
    eta = apparent_viscosity(shear_rates, K_fixed, n)
    label = f'n = {n}' + (' (Newtonian)' if n == 1.0 else '')
    ax3.loglog(shear_rates, eta * 1000, linewidth=3, color=color, label=label)

ax3.set_xlabel('Shear Rate, γ̇ (1/s)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Apparent Viscosity, η (mPa·s)', fontsize=12, fontweight='bold')
ax3.set_title('Effect of Flow Behavior Index (n)\nK = 1 Pa·sⁿ fixed',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3, which='both')

# Add annotations
ax3.annotate('Stronger\nshear-thinning', xy=(100, 0.01), fontsize=11,
            fontweight='bold', ha='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

# Plot 4: Effect of yield stress (Herschel-Bulkley)
tau_y_values = [0, 20, 50, 100, 200]  # Pa
K_HB = 2.0
n_HB = 0.5

for tau_y, color in zip(tau_y_values, colors_n):
    if tau_y == 0:
        tau = power_law_model(shear_rates, K_HB, n_HB)
        label = 'τ_y = 0 (Power-law)'
    else:
        tau = herschel_bulkley_model(shear_rates, tau_y, K_HB, n_HB)
        label = f'τ_y = {tau_y} Pa'
    
    ax4.loglog(shear_rates, tau, linewidth=3, color=color, label=label)

ax4.set_xlabel('Shear Rate, γ̇ (1/s)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Shear Stress, τ (Pa)', fontsize=12, fontweight='bold')
ax4.set_title('Effect of Yield Stress\n(Herschel-Bulkley Model)',
             fontsize=13, fontweight='bold')
ax4.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax4.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("\n💡 Rheological Model Insights:")
print("   • Newtonian: constant viscosity (rare for bioinks)")
print("   • Power-law: shear-thinning when n < 1")
print("   • Lower n → stronger shear-thinning (better for cells)")
print("   • Herschel-Bulkley: adds yield stress for shape fidelity")
print("   • Yield stress: stress needed to initiate flow")
print("   • High τ_y: good shape retention, harder to extrude")
print("   • Ideal bioink: high η at rest, low η during printing")

## Part 3: Thixotropy and Recovery Analysis

In [ ]:
# Simulate thixotropic recovery
time_recovery = np.linspace(0, 300, 500)  # seconds

# Different bioink recovery profiles
recovery_profiles = {
    'Fast (ideal)': {'tau_i': 10, 'tau_f': 200, 'time_const': 10, 'color': '#2ecc71'},
    'Moderate': {'tau_i': 10, 'tau_f': 150, 'time_const': 30, 'color': '#3498db'},
    'Slow': {'tau_i': 10, 'tau_f': 120, 'time_const': 80, 'color': '#f39c12'},
    'Very slow': {'tau_i': 10, 'tau_f': 100, 'time_const': 150, 'color': '#e74c3c'}
}

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Recovery curves
for profile_name, props in recovery_profiles.items():
    recovery_curve = thixotropic_recovery(time_recovery, props['tau_i'],
                                         props['tau_f'], props['time_const'])
    ax1.plot(time_recovery, recovery_curve, linewidth=3, color=props['color'],
            label=profile_name)
    
    # Mark 90% recovery time
    target = 0.9 * (props['tau_f'] - props['tau_i']) + props['tau_i']
    t_90 = props['time_const'] * np.log(10)  # Time to 90% recovery
    if t_90 < 300:
        ax1.plot(t_90, target, 'o', markersize=10, color=props['color'],
                markeredgecolor='black', markeredgewidth=2)

ax1.axhline(100, color='green', linestyle='--', linewidth=2,
           label='Target yield stress')
ax1.set_xlabel('Time After Shear Cessation (s)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Yield Stress, τ_y (Pa)', fontsize=12, fontweight='bold')
ax1.set_title('Thixotropic Recovery\nτ(t) = τ_∞ - (τ_∞-τ₀)·exp(-t/λ)',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True)
ax1.grid(True, alpha=0.3)

# Plot 2: Time to 90% recovery
recovery_times = [props['time_const'] * np.log(10) 
                 for props in recovery_profiles.values()]
profile_names = list(recovery_profiles.keys())
colors_recovery = [props['color'] for props in recovery_profiles.values()]

bars = ax2.barh(profile_names, recovery_times, color=colors_recovery,
               alpha=0.7, edgecolor='black', linewidth=2)

# Add value labels
for bar, value in zip(bars, recovery_times):
    ax2.text(value + 5, bar.get_y() + bar.get_height()/2,
            f'{value:.0f} s',
            va='center', fontweight='bold', fontsize=11)

# Add target range
ax2.axvspan(0, 60, alpha=0.2, color='green', label='Ideal (<60 s)')
ax2.axvspan(60, 120, alpha=0.2, color='yellow', label='Acceptable')

ax2.set_xlabel('Time to 90% Recovery (s)', fontsize=12, fontweight='bold')
ax2.set_title('Recovery Time Comparison', fontsize=13, fontweight='bold')
ax2.legend(loc='lower right', frameon=True, fancybox=True)
ax2.grid(True, alpha=0.3, axis='x')

# Plot 3: 3-interval thixotropy test simulation
# Interval 1: Low shear (0.1 1/s) for 60s
# Interval 2: High shear (100 1/s) for 60s  
# Interval 3: Low shear (0.1 1/s) for 180s

time_total = np.concatenate([
    np.linspace(0, 60, 100),
    np.linspace(60, 120, 100),
    np.linspace(120, 300, 200)
])

# Simulate viscosity response
viscosity_response = []
for t in time_total:
    if t < 60:
        # Interval 1: at rest
        eta = 100 - 20 * (1 - np.exp(-t/10))  # Initial structure buildup
    elif t < 120:
        # Interval 2: high shear (structure breakdown)
        t_rel = t - 60
        eta = 2 + 78 * np.exp(-t_rel/5)  # Rapid breakdown
    else:
        # Interval 3: recovery
        t_rel = t - 120
        eta = 100 - 98 * np.exp(-t_rel/30)  # Recovery
    
    viscosity_response.append(eta)

ax3.plot(time_total, viscosity_response, linewidth=3, color='#3498db')

# Mark intervals
ax3.axvspan(0, 60, alpha=0.15, color='green', label='Low shear (0.1 1/s)')
ax3.axvspan(60, 120, alpha=0.15, color='red', label='High shear (100 1/s)')
ax3.axvspan(120, 300, alpha=0.15, color='green')

ax3.axvline(60, color='black', linestyle='--', linewidth=2)
ax3.axvline(120, color='black', linestyle='--', linewidth=2)

ax3.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Apparent Viscosity (Pa·s)', fontsize=12, fontweight='bold')
ax3.set_title('3-Interval Thixotropy Test\n(Breakdown & Recovery)',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3)

# Plot 4: Shape fidelity over time
# Simulate printed layer collapse
time_print = np.linspace(0, 300, 500)
layer_height_initial = 0.5  # mm

# Different recovery scenarios
for profile_name, props in recovery_profiles.items():
    # Recovery affects collapse rate
    # Faster recovery → less collapse
    collapse_rate = 0.1 / props['time_const']  # 1/s
    
    # Asymptotic collapse model
    final_collapse = 0.3 * (1 - props['tau_f']/200)  # More yield stress → less collapse
    layer_height = layer_height_initial - final_collapse * (1 - np.exp(-collapse_rate * time_print))
    
    retention = (layer_height / layer_height_initial) * 100
    ax4.plot(time_print, retention, linewidth=3, color=props['color'],
            label=profile_name)

ax4.axhline(90, color='green', linestyle='--', linewidth=2,
           label='Target (>90%)')
ax4.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Shape Retention (%)', fontsize=12, fontweight='bold')
ax4.set_title('Shape Fidelity After Printing\n(Collapse Prevention)',
             fontsize=13, fontweight='bold')
ax4.legend(loc='best', frameon=True, fancybox=True)
ax4.grid(True, alpha=0.3)
ax4.set_ylim(70, 102)

plt.tight_layout()
plt.show()

print("\n💡 Thixotropy Insights:")
print("   • Thixotropy: time-dependent viscosity recovery")
print("   • Fast recovery (<60 s): ideal for bioprinting")
print("   • Structure breaks down under shear (printing)")
print("   • Must rebuild quickly after deposition")
print("   • Slow recovery → shape collapse → poor fidelity")
print("   • 3-interval test: standard rheological characterization")
print("   • Faster recovery → better multi-layer printing")

## Part 4: Interactive Bioink Rheology Analyzer

### 🎯 STUDENT TASK 1: Characterize Your Bioink

**Input rheological properties and get printability assessment:**

In [ ]:
# ==========================================
# STUDENT PARAMETERS - MODIFY THESE VALUES
# ==========================================

# Bioink name
bioink_name = 'Alginate-Gelatin'

# Power-law parameters (from rheometer measurement)
consistency_index = 3.5       # K (Pa·sⁿ) - try: 0.5, 2.0, 5.0
flow_behavior_index = 0.45    # n (dimensionless) - try: 0.3, 0.5, 0.7

# Yield stress (from oscillatory rheology)
yield_stress_Pa = 80          # Pa - try: 20, 50, 100, 200

# Recovery kinetics
recovery_time_90 = 45         # seconds to 90% recovery - try: 15, 30, 60, 120

# Bioink properties
bioink_density = 1050         # kg/m³

# ==========================================

# Calculate derived properties
shear_rates_analysis = np.logspace(-1, 3, 100)

# Viscosity at rest (0.1 1/s) and during printing (100 1/s)
eta_rest = apparent_viscosity(0.1, consistency_index, flow_behavior_index)
eta_print = apparent_viscosity(100, consistency_index, flow_behavior_index)

# Calculate recovery time constant from 90% time
recovery_time_const = recovery_time_90 / np.log(10)

# Printability assessment
assessment = assess_printability_extrusion(eta_rest, eta_print,
                                          yield_stress_Pa, recovery_time_90)

# Printability number for different heights
heights_mm = [1, 5, 10, 20]  # mm
Pr_values = [calculate_printability_number(yield_stress_Pa, bioink_density, h*1e-3)
            for h in heights_mm]

# Create visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Viscosity Profile', 'Printability Score',
                   'Shape Fidelity Prediction', 'Rheological Summary'),
    specs=[[{'type': 'scatter'}, {'type': 'indicator'}],
           [{'type': 'bar'}, {'type': 'table'}]]
)

# Plot 1: Viscosity curve
eta_profile = apparent_viscosity(shear_rates_analysis, consistency_index,
                                flow_behavior_index)

fig.add_trace(
    go.Scatter(x=shear_rates_analysis, y=eta_profile*1000,
              mode='lines', name='Viscosity',
              line=dict(color='blue', width=3)),
    row=1, col=1
)

# Mark key points
fig.add_trace(
    go.Scatter(x=[0.1, 100], y=[eta_rest*1000, eta_print*1000],
              mode='markers', name='Key points',
              marker=dict(size=15, color='red', symbol='star')),
    row=1, col=1
)

fig.update_xaxes(title_text="Shear Rate (1/s)", type="log", row=1, col=1)
fig.update_yaxes(title_text="Viscosity (mPa·s)", type="log", row=1, col=1)

# Plot 2: Printability gauge
score = assessment['score']
rating = assessment['rating']

if rating == 'Excellent':
    gauge_color = "green"
elif rating == 'Good':
    gauge_color = "lightgreen"
elif rating == 'Marginal':
    gauge_color = "yellow"
else:
    gauge_color = "red"

fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=score,
        title={'text': f"Printability<br>{rating}"},
        gauge={
            'axis': {'range': [None, 100]},
            'bar': {'color': gauge_color},
            'steps': [
                {'range': [0, 40], 'color': "lightgray"},
                {'range': [40, 60], 'color': "lightyellow"},
                {'range': [60, 80], 'color': "lightgreen"},
                {'range': [80, 100], 'color': "green"}],
            'threshold': {
                'line': {'color': "darkgreen", 'width': 4},
                'thickness': 0.75,
                'value': 80}
        }),
    row=1, col=2
)

# Plot 3: Printability number for different heights
colors_Pr = ['green' if Pr > 5 else 'yellow' if Pr > 1 else 'red'
            for Pr in Pr_values]

fig.add_trace(
    go.Bar(x=[f'{h} mm' for h in heights_mm], y=Pr_values,
          marker_color=colors_Pr,
          text=[f'{Pr:.1f}' for Pr in Pr_values],
          textposition='outside'),
    row=2, col=1
)

# Add threshold line manually for Plotly compatibility
fig.add_trace(
    go.Scatter(x=[0, len(heights_mm)-1], y=[1, 1],
              mode='lines', name='Pr=1 (marginal)',
              line=dict(color='red', width=2, dash='dash'),
              showlegend=False),
    row=2, col=1
)

fig.update_yaxes(title_text="Printability Number (Pr)", row=2, col=1)

# Plot 4: Summary table
fig.add_trace(
    go.Table(
        header=dict(values=['Property', 'Value'],
                   fill_color='paleturquoise',
                   align='left',
                   font=dict(size=11)),
        cells=dict(values=[
            ['K (Consistency)', 'n (Flow Index)', 'Yield Stress',
             'η at Rest (0.1 1/s)', 'η at Print (100 1/s)',
             'Shear-Thinning Ratio', 'Recovery Time (90%)', 'Score'],
            [f'{consistency_index:.2f} Pa·sⁿ', f'{flow_behavior_index:.2f}',
             f'{yield_stress_Pa} Pa', f'{eta_rest*1000:.0f} mPa·s',
             f'{eta_print*1000:.1f} mPa·s',
             f'{eta_rest/eta_print:.0f}×',
             f'{recovery_time_90} s', f'{score}/100']
        ],
        fill_color='lavender',
        align='left',
        font=dict(size=10))),
    row=2, col=2
)

fig.update_layout(height=900, showlegend=False,
                 title_text=f"Bioink Rheology Analysis - {bioink_name}")
fig.show()

# Detailed text output
print("="*70)
print("BIOINK RHEOLOGY ANALYSIS")
print("="*70)
print(f"\nBioink: {bioink_name}")

print(f"\nPower-Law Parameters:")
print(f"  • Consistency index (K): {consistency_index:.2f} Pa·sⁿ")
print(f"  • Flow behavior index (n): {flow_behavior_index:.2f}")
print(f"  • Yield stress (τ_y): {yield_stress_Pa} Pa")

print(f"\nViscosity Analysis:")
print(f"  • At rest (0.1 1/s): {eta_rest*1000:.0f} mPa·s")
print(f"  • During printing (100 1/s): {eta_print*1000:.1f} mPa·s")
print(f"  • Shear-thinning ratio: {eta_rest/eta_print:.0f}×")

print(f"\nRecovery Kinetics:")
print(f"  • Time to 90% recovery: {recovery_time_90} s")
print(f"  • Time constant: {recovery_time_const:.1f} s")

print(f"\nPrintability Assessment:")
print(f"  • Overall score: {score}/100")
print(f"  • Rating: {rating}")

if assessment['issues']:
    print(f"\n  Issues:")
    for issue in assessment['issues']:
        print(f"    ⚠️  {issue}")

if assessment['recommendations']:
    print(f"\n  Recommendations:")
    for rec in assessment['recommendations']:
        print(f"    💡 {rec}")

print(f"\nShape Fidelity (Printability Number):")
for h, Pr in zip(heights_mm, Pr_values):
    if Pr > 5:
        status = "✅ Excellent"
    elif Pr > 1:
        status = "✓ Acceptable"
    else:
        status = "❌ Poor - will collapse"
    print(f"  • {h} mm height: Pr = {Pr:.1f} ({status})")

print(f"\nInterpretation:")
if flow_behavior_index < 0.5:
    print("  • Strong shear-thinning (n < 0.5) - excellent for cell protection")
elif flow_behavior_index < 0.7:
    print("  • Moderate shear-thinning - good balance")
else:
    print("  • Weak shear-thinning - consider formulation optimization")

if yield_stress_Pa > 100:
    print("  • High yield stress - excellent shape retention")
elif yield_stress_Pa > 50:
    print("  • Moderate yield stress - good for moderate heights")
else:
    print("  • Low yield stress - may need crosslinking support")

if recovery_time_90 < 60:
    print("  • Fast recovery - ideal for multi-layer printing")
elif recovery_time_90 < 120:
    print("  • Moderate recovery - acceptable with proper timing")
else:
    print("  • Slow recovery - consider adding crosslinking agent")

print("\n" + "="*70)

## Part 5: Printability Window Design

In [ ]:
# Create design space for bioink formulation
concentrations = np.linspace(1, 10, 50)  # % w/v
n_values_space = np.linspace(0.3, 0.9, 50)

C_grid, N_grid = np.meshgrid(concentrations, n_values_space)

# Model how K scales with concentration (empirical)
# K ≈ K_0 · C^α, where α ≈ 3-4 for polymers
K_0 = 0.01
alpha_K = 3.5

# Calculate metrics
eta_rest_grid = np.zeros_like(C_grid)
eta_print_grid = np.zeros_like(C_grid)
ratio_grid = np.zeros_like(C_grid)

for i in range(C_grid.shape[0]):
    for j in range(C_grid.shape[1]):
        C = C_grid[i, j]
        n = N_grid[i, j]
        
        K = K_0 * (C ** alpha_K)
        
        eta_rest = apparent_viscosity(0.1, K, n)
        eta_print = apparent_viscosity(100, K, n)
        
        eta_rest_grid[i, j] = eta_rest * 1000  # mPa·s
        eta_print_grid[i, j] = eta_print * 1000
        ratio_grid[i, j] = eta_rest / eta_print

# Create visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 14))

# Plot 1: Viscosity at rest
contour1 = ax1.contourf(C_grid, N_grid, eta_rest_grid, levels=20, cmap='viridis')
contour1_lines = ax1.contour(C_grid, N_grid, eta_rest_grid,
                             levels=[100, 1000, 10000], colors='white', linewidths=2)
ax1.clabel(contour1_lines, inline=True, fontsize=10, fmt='%d mPa·s')
cbar1 = plt.colorbar(contour1, ax=ax1)
cbar1.set_label('Viscosity at Rest (mPa·s)', fontsize=11, fontweight='bold')

ax1.set_xlabel('Concentration (% w/v)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Flow Behavior Index (n)', fontsize=12, fontweight='bold')
ax1.set_title('Viscosity at Rest (0.1 1/s)',
             fontsize=13, fontweight='bold')

# Plot 2: Viscosity during printing
contour2 = ax2.contourf(C_grid, N_grid, eta_print_grid, levels=20, cmap='plasma')
contour2_lines = ax2.contour(C_grid, N_grid, eta_print_grid,
                             levels=[0.1, 1, 10], colors='white', linewidths=2)
ax2.clabel(contour2_lines, inline=True, fontsize=10, fmt='%.1f mPa·s')
cbar2 = plt.colorbar(contour2, ax=ax2)
cbar2.set_label('Viscosity During Printing (mPa·s)', fontsize=11, fontweight='bold')

ax2.set_xlabel('Concentration (% w/v)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Flow Behavior Index (n)', fontsize=12, fontweight='bold')
ax2.set_title('Viscosity During Printing (100 1/s)',
             fontsize=13, fontweight='bold')

# Plot 3: Shear-thinning ratio
contour3 = ax3.contourf(C_grid, N_grid, ratio_grid, levels=20, cmap='RdYlGn')
contour3_lines = ax3.contour(C_grid, N_grid, ratio_grid,
                             levels=[10, 100, 1000], colors='black', linewidths=2)
ax3.clabel(contour3_lines, inline=True, fontsize=10, fmt='%d×')
cbar3 = plt.colorbar(contour3, ax=ax3)
cbar3.set_label('Shear-Thinning Ratio', fontsize=11, fontweight='bold')

ax3.set_xlabel('Concentration (% w/v)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Flow Behavior Index (n)', fontsize=12, fontweight='bold')
ax3.set_title('Shear-Thinning Ratio (η_rest/η_print)',
             fontsize=13, fontweight='bold')

# Plot 4: Printability window
# Define constraints:
# 1. η_rest: 100-10000 mPa·s
# 2. η_print: 0.1-10 mPa·s
# 3. ratio > 10

printable = (eta_rest_grid >= 100) & (eta_rest_grid <= 10000) & \
           (eta_print_grid >= 0.1) & (eta_print_grid <= 10) & \
           (ratio_grid >= 10)

ax4.contourf(C_grid, N_grid, printable.astype(float),
            levels=[0.5, 1.5], colors=['white', 'lightgreen'], alpha=0.7)

# Add constraint boundaries
ax4.contour(C_grid, N_grid, eta_rest_grid, levels=[100, 10000],
           colors=['blue'], linewidths=2, linestyles='dashed')
ax4.contour(C_grid, N_grid, eta_print_grid, levels=[0.1, 10],
           colors=['red'], linewidths=2, linestyles='dashed')

# Mark example formulations
examples = [
    {'name': 'Optimal', 'C': 4, 'n': 0.45, 'color': 'green'},
    {'name': 'High Conc', 'C': 7, 'n': 0.5, 'color': 'orange'},
    {'name': 'Low n', 'C': 5, 'n': 0.35, 'color': 'blue'}
]

for ex in examples:
    ax4.plot(ex['C'], ex['n'], 'o', markersize=15, color=ex['color'],
            markeredgecolor='black', markeredgewidth=2)
    ax4.annotate(ex['name'], xy=(ex['C'], ex['n']), xytext=(10, 10),
                textcoords='offset points', fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor=ex['color'],
                         alpha=0.6, edgecolor='black'))

ax4.set_xlabel('Concentration (% w/v)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Flow Behavior Index (n)', fontsize=12, fontweight='bold')
ax4.set_title('Printability Window\n(Green = All constraints satisfied)',
             fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Printability Window Insights:")
print("   • Higher concentration → higher viscosity at all shear rates")
print("   • Lower n → stronger shear-thinning → larger ratio")
print("   • Sweet spot: 3-6% concentration with n = 0.4-0.6")
print("   • Too high concentration → difficulty extruding")
print("   • Too low n → may be difficult to achieve practically")
print("   • Multiple formulations can work - choose based on cell type")
print("   • Always verify with experimental rheology!")

## Part 6: Summary and Key Takeaways

In [ ]:
print("="*80)
print("CHAPTER 4 EXERCISE 7: KEY LEARNING POINTS")
print("="*80)
print("""
1. RHEOLOGY DETERMINES PRINTABILITY
   → Ideal bioink behavior:
     • HIGH viscosity at rest (shape retention)
     • LOW viscosity during printing (easy flow)
     • RAPID recovery after deposition
   → This requires shear-thinning + yield stress + thixotropy

2. POWER-LAW MODEL DESCRIBES SHEAR-THINNING
   → τ = K·γ̇ⁿ
   → n < 1: shear-thinning (pseudoplastic)
   → Lower n → stronger thinning
   → Typical for bioinks: n = 0.3-0.7
   → η_apparent = K·γ̇^(n-1) decreases with shear rate

3. HERSCHEL-BULKLEY ADDS YIELD STRESS
   → τ = τ_y + K·γ̇ⁿ
   → τ_y: minimum stress to initiate flow
   → Essential for shape fidelity and self-support
   → Target for extrusion: 50-200 Pa
   → Too low → collapse; too high → clogging

4. PRINTABILITY NUMBER PREDICTS SHAPE FIDELITY
   → Pr = τ_y / (ρ·g·h)
   → Pr >> 1: excellent retention
   → Pr ≈ 1: marginal stability
   → Pr << 1: structure collapses
   → Pr > 5 recommended for multi-layer constructs

5. SHEAR-THINNING PROTECTS CELLS
   → High viscosity → high shear stress → cell damage
   → Shear-thinning reduces stress during extrusion
   → Can reduce stress by 40-70% vs Newtonian
   → Enables printing of cell-laden bioinks
   → n = 0.4-0.5 ideal balance

6. THIXOTROPY ENABLES RAPID RECOVERY
   → Time-dependent structure breakdown and rebuilding
   → Structure breaks during shear (printing)
   → Must recover quickly after deposition (<60 s ideal)
   → Slower recovery → spreading, poor fidelity
   → 3-interval test: standard characterization

7. VISCOSITY REQUIREMENTS ARE TECHNIQUE-SPECIFIC
   → Extrusion:
     • At rest: 100-10,000 mPa·s
     • During print: 0.1-10 mPa·s
     • Ratio: 10-1000×
   → Inkjet:
     • 1-10 mPa·s (nearly Newtonian)
     • Z-number: 1 < Z < 10
   → Light-based: depends on photoresin

8. CONCENTRATION AFFECTS ALL PROPERTIES
   → Higher concentration:
     ✓ Higher yield stress
     ✓ Better shape retention
     ✗ Higher printing pressure
     ✗ Lower cell density possible
   → Optimal range: 2-6% for most hydrogels

9. PRACTICAL CHARACTERIZATION METHODS
   → Flow sweep: measure τ vs γ̇ (get K, n)
   → Oscillatory: measure G', G" (get τ_y from crossover)
   → 3-interval thixotropy: measure recovery kinetics
   → Temperature sweep: check gelation behavior
   → Always test with cells for validation!

10. FORMULATION STRATEGIES
    → To increase shear-thinning:
      • Add high MW polymer
      • Increase concentration
      • Add nanoparticles (nanocellulose, clay)
    → To increase yield stress:
      • Add crosslinking agent
      • Increase polymer MW
      • Add nanofibers
    → To improve recovery:
      • Add rapid crosslinking (Ca²⁺, light)
      • Use supramolecular interactions
      • Combine multiple mechanisms
""")
print("="*80)

## 🎓 REFLECTION QUESTIONS

### Question 1
**For a power-law fluid with K = 2 Pa·sⁿ and n = 0.5, calculate the apparent viscosity at shear rates of 0.1, 10, and 100 1/s. By what factor does viscosity decrease from rest to printing?**

### Question 2  
**A bioink has yield stress τ_y = 100 Pa and density 1050 kg/m³. Calculate the printability number for heights of 2, 5, and 10 mm. At what height does the structure become unstable (Pr < 1)?**

### Question 3
**Explain physically why shear-thinning occurs in polymer solutions. Relate your explanation to the molecular structure and entanglements.**

### Question 4
**You measure a bioink: η = 5000 mPa·s at 0.1 1/s and η = 50 mPa·s at 100 1/s. Assuming power-law behavior, calculate K and n. Is this bioink suitable for extrusion?**

### Question 5
**Compare three bioinks for cardiac tissue printing: (A) n=0.3, τ_y=150 Pa, t_90=30s; (B) n=0.6, τ_y=80 Pa, t_90=15s; (C) n=0.4, τ_y=200 Pa, t_90=90s. Which would you choose and why? Consider shape fidelity, cell protection, and layer-by-layer printing.**

## 📚 Additional Challenges (Optional)

### Challenge 1: Cross-Model Analysis
Research the Cross model for viscosity (another shear-thinning model). How does it compare to power-law? When is each more appropriate?

### Challenge 2: Temperature-Dependent Rheology
Model how viscosity changes with temperature for a thermosensitive bioink (e.g., gelatin, Pluronic). Design a printing protocol that exploits this.

### Challenge 3: Composite Bioinks
How would you predict the rheology of a composite bioink (e.g., alginate + nanocellulose)? Research mixing rules for complex fluids.

### Challenge 4: In-Situ Crosslinking
Model how ionic crosslinking (e.g., Ca²⁺ for alginate) affects yield stress evolution during printing. How fast must crosslinking occur?

## 🎯 Congratulations!

You've completed Exercise 7: Bioink Rheology and Printability!

**You now understand:**
- ✓ Rheological models (Newtonian, power-law, Herschel-Bulkley)
- ✓ Shear-thinning behavior and flow behavior index
- ✓ Yield stress and shape fidelity (Printability number)
- ✓ Thixotropy and recovery kinetics
- ✓ Printability windows for different techniques
- ✓ Formulation strategies for optimal bioink design

**Next Steps:**
- Continue to Exercise 8: Integrated Bioprinting Process Design
- Review Chapter 4.2 for bioink rheology fundamentals
- Explore research papers on advanced bioink formulations

---

*Exercise created for Biofabrication Chapter 4 - Master's Level Bioengineering*